In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.stats import gaussian_kde
import cvxpy as cp
import pandas as pd

# Use dark background
plt.style.use('dark_background')

/Users/macbook/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/macbook/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Custom loss function
def compute_loss(X_0, X_1, a, b, t):
    eps = 1e-6

    # True Positives Lost (Class 1 points classified as Class 0)
    fn = np.sum(a * X_1[:, 0] + b * X_1[:, 1] < t)

    # True Positives Accepted (Class 1 points classified as Class 1)
    tp = np.sum(a * X_1[:, 0] + b * X_1[:, 1] >= t-eps)
    
    # True Negatives Accepted (Class 0 points classified as Class 1)
    fp = np.sum(a * X_0[:, 0] + b * X_0[:, 1] >= t-eps)

    # True Negatives Lost (Class 0 points classified as Class 0)
    tn = np.sum(a * X_0[:, 0] + b * X_0[:, 1] < t)
    
    # Loss is the sum of the penalties
    return fn, tp, fp, tn

# Calculate the movement vector for each selected point
def move_towards_boundary(X, mask, a, b, t):
    norm_factor = np.sqrt(a**2 + b**2)
    movement = np.outer((a * X[mask, 0] + b * X[mask, 1] - t) / norm_factor, np.array([a, b]) / norm_factor)
    X[mask] = X[mask] - movement

def plot_for(X_0, X_1, a, b, t, B, mode='NB', gamma=0.6):
    if mode == 'NB':
        # Calculate initial loss
        tp_lost, tp_accepted, tn_accepted, tn_lost = compute_loss(X_0, X_1, a, b, t)
        firm_utility = tp_accepted - tn_accepted
        loss_initial = tp_lost + tn_lost

        # Movement region parameters
        mask_0 = (t - B <= a * X_0[:, 0] + b * X_0[:, 1]) & (a * X_0[:, 0] + b * X_0[:, 1] < t)
        mask_1 = (t - B <= a * X_1[:, 0] + b * X_1[:, 1]) & (a * X_1[:, 0] + b * X_1[:, 1] < t)

        # Move the distributions
        X_0_moved = X_0.copy()
        X_1_moved = X_1.copy()

        move_towards_boundary(X_0_moved, mask_0, a, b, t)
        move_towards_boundary(X_1_moved, mask_1, a, b, t)

        # Calculate new loss after movement
        tp_lost_moved, tp_accepted_moved, tn_accepted_moved, tn_lost_moved = compute_loss(X_0_moved, X_1_moved, a, b, t)
        firm_utility_moved = tp_accepted_moved - tn_accepted_moved
        loss_new = tp_lost_moved + tn_lost_moved

        # Create the plot
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))

        x_min = np.min([X_0[:, 0], X_1[:, 0], X_0_moved[:, 0], X_1_moved[:, 0]])
        x_max = np.max([X_0[:, 0], X_1[:, 0], X_0_moved[:, 0], X_1_moved[:, 0]])
        y_min = np.min([X_0[:, 1], X_1[:, 1], X_0_moved[:, 1], X_1_moved[:, 1]])
        y_max = np.max([X_0[:, 1], X_1[:, 1], X_0_moved[:, 1], X_1_moved[:, 1]])

        # Plot 1: Initial Distribution
        ax[0].scatter(X_0[:, 0], X_0[:, 1], alpha=0.3, color='red', label='Class 0')
        ax[0].scatter(X_1[:, 0], X_1[:, 1], alpha=0.3, color='blue', label='Class 1')
        ax[0].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[0].set_xlim(x_min, x_max)
        ax[0].set_ylim(y_min, y_max)
        ax[0].set_title(f'Initial Distribution\n (FP+FN): {loss_initial:.2f} | (TP+TN): {firm_utility:.2f} | (TP-FP) : {tp_accepted-tp_lost}')
        ax[0].set_xlabel('Feature 1')
        ax[0].set_ylabel('Feature 2')
        ax[0].legend()

        # Plot 2: Distribution After Movement
        ax[1].scatter(X_0_moved[:, 0], X_0_moved[:, 1], alpha=0.3, color='red', label='Class 0')
        ax[1].scatter(X_1_moved[:, 0], X_1_moved[:, 1], alpha=0.3, color='blue', label='Class 1')
        ax[1].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[1].set_xlim(x_min, x_max)
        ax[1].set_ylim(y_min, y_max)
        ax[1].set_title(f'After Movement\n (FP+FN): {loss_new:.2f} | (TP+TN): {firm_utility_moved:.2f} | (TP-FP) : {tp_accepted_moved-tp_lost_moved}')
        ax[1].set_xlabel('Feature 1')
        ax[1].set_ylabel('Feature 2')
        ax[1].legend()

        plt.tight_layout()
        plt.show()

    elif mode == 'B':
        if a > b:
            a_B = w(a, gamma)
            b_B = 1-a_B
        else:
            b_B = w(b, gamma)
            a_B = 1-b_B
    
        # Calculate initial loss
        tp_lost, tp_accepted, tn_accepted, tn_lost = compute_loss(X_0, X_1, a, b, t)
        firm_utility = tp_accepted - tn_accepted
        loss_initial = tp_lost + tn_lost

        # Movement region parameters
        mask_0 = (t - B <= a_B * X_0[:, 0] + b_B * X_0[:, 1]) & (a_B * X_0[:, 0] + b_B * X_0[:, 1] < t)
        mask_1 = (t - B <= a_B * X_1[:, 0] + b_B * X_1[:, 1]) & (a_B * X_1[:, 0] + b_B * X_1[:, 1] < t)

        # Move the distributions
        X_0_moved = X_0.copy()
        X_1_moved = X_1.copy()

        move_towards_boundary(X_0_moved, mask_0, a_B, b_B, t)
        move_towards_boundary(X_1_moved, mask_1, a_B, b_B, t)

        # Calculate new loss after movement
        tp_lost_moved, tp_accepted_moved, tn_accepted_moved, tn_lost_moved = compute_loss(X_0_moved, X_1_moved, a, b, t)
        firm_utility_moved = tp_accepted_moved - tn_accepted_moved
        loss_new = tp_lost_moved + tn_lost_moved

        # Create the plot
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))

        x_min = np.min([X_0[:, 0], X_1[:, 0], X_0_moved[:, 0], X_1_moved[:, 0]])
        x_max = np.max([X_0[:, 0], X_1[:, 0], X_0_moved[:, 0], X_1_moved[:, 0]])
        y_min = np.min([X_0[:, 1], X_1[:, 1], X_0_moved[:, 1], X_1_moved[:, 1]])
        y_max = np.max([X_0[:, 1], X_1[:, 1], X_0_moved[:, 1], X_1_moved[:, 1]])

        # Plot 1: Initial Distribution
        ax[0].scatter(X_0[:, 0], X_0[:, 1], alpha=0.3, color='red', label='Class 0')
        ax[0].scatter(X_1[:, 0], X_1[:, 1], alpha=0.3, color='blue', label='Class 1')
        ax[0].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[0].plot(np.linspace(x_min, x_max, 100), (t - a_B * np.linspace(x_min, x_max, 100)) / b_B, color='gold', linestyle='--', label='Perceived Boundary')
        ax[0].set_xlim(x_min, x_max)
        ax[0].set_ylim(y_min, y_max)
        ax[0].set_title(f'Initial Distribution\n (FP+FN): {loss_initial:.2f} | (TP+TN): {firm_utility:.2f} | (TP-FP) : {tp_accepted-tp_lost}')
        ax[0].set_xlabel('Feature 1')
        ax[0].set_ylabel('Feature 2')
        ax[0].legend()

        # Plot 2: Distribution After Movement
        ax[1].scatter(X_0_moved[:, 0], X_0_moved[:, 1], alpha=0.3, color='red', label='Class 0')
        ax[1].scatter(X_1_moved[:, 0], X_1_moved[:, 1], alpha=0.3, color='blue', label='Class 1')
        ax[1].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[1].plot(np.linspace(x_min, x_max, 100), (t - a_B * np.linspace(x_min, x_max, 100)) / b_B, color='gold', linestyle='--', label='Perceived Boundary')
        ax[1].set_xlim(x_min, x_max)
        ax[1].set_ylim(y_min, y_max)
        ax[1].set_title(f'After Movement\n (FP+FN): {loss_new:.2f} | (TP+TN): {firm_utility_moved:.2f} | (TP-FP) : {tp_accepted_moved-tp_lost_moved}')
        ax[1].set_xlabel('Feature 1')
        ax[1].set_ylabel('Feature 2')
        ax[1].legend()

        plt.tight_layout()
        plt.show()



def plot_dist(X_0, X_1, a, b, t, B, mode='NB', gamma=0.6):
    if mode == 'NB':
        # Calculate initial loss
        tp_lost, tp_accepted, tn_accepted, tn_lost = compute_loss(X_0, X_1, a, b, t)
        firm_utility = tp_accepted - tn_accepted
        loss_initial = tp_lost + tn_lost

        # Movement region parameters
        mask_0 = (t - B <= a * X_0[:, 0] + b * X_0[:, 1]) & (a * X_0[:, 0] + b * X_0[:, 1] < t)
        mask_1 = (t - B <= a * X_1[:, 0] + b * X_1[:, 1]) & (a * X_1[:, 0] + b * X_1[:, 1] < t)

        # Move the distributions
        X_0_moved = X_0.copy()
        X_1_moved = X_1.copy()

        move_towards_boundary(X_0_moved, mask_0, a, b, t)
        move_towards_boundary(X_1_moved, mask_1, a, b, t)

        # Calculate new loss after movement
        tp_lost_moved, tp_accepted_moved, tn_accepted_moved, tn_lost_moved = compute_loss(X_0_moved, X_1_moved, a, b, t)
        firm_utility_moved = tp_accepted_moved - tn_accepted_moved
        loss_new = tp_lost_moved + tn_lost_moved

        # Create a grid of points to evaluate the KDE
        x_min, x_max = min(np.min(X_0[:, 0]), np.min(X_1[:, 0])) - 1, max(np.max(X_0[:, 0]), np.max(X_1[:, 0])) + 1
        y_min, y_max = min(np.min(X_0[:, 1]), np.min(X_1[:, 1])) - 1, max(np.max(X_0[:, 1]), np.max(X_1[:, 1])) + 1
        xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
        positions = np.vstack([xx.ravel(), yy.ravel()])

        # KDE for each class
        kde_0 = gaussian_kde(X_0.T)
        kde_1 = gaussian_kde(X_1.T)
        kde_0_moved = gaussian_kde(X_0_moved.T)
        kde_1_moved = gaussian_kde(X_1_moved.T)

        # Evaluate the KDE on the grid
        f_0 = np.reshape(kde_0(positions).T, xx.shape)
        f_1 = np.reshape(kde_1(positions).T, xx.shape)
        f_0_moved = np.reshape(kde_0_moved(positions).T, xx.shape)
        f_1_moved = np.reshape(kde_1_moved(positions).T, xx.shape)

        # Create the plot
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))

        # Plot 1: Initial Distribution KDE
        ax[0].contourf(xx, yy, f_0, cmap='Reds', alpha=0.4)
        ax[0].contourf(xx, yy, f_1, cmap='Blues', alpha=0.4)
        ax[0].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[0].set_title(f'Initial Distribution\n (FN+FP): {loss_initial} | (TP+TN): {firm_utility} | (TP-FP) : {tp_accepted-tp_lost}')
        ax[0].set_xlim(x_min, x_max)
        ax[0].set_ylim(y_min, y_max)
        ax[0].set_xlabel('Feature 1')
        ax[0].set_ylabel('Feature 2')
        ax[0].legend()

        # Plot 2: Distribution After Movement KDE
        ax[1].contourf(xx, yy, f_0_moved, cmap='Reds', alpha=0.4)
        ax[1].contourf(xx, yy, f_1_moved, cmap='Blues', alpha=0.4)
        ax[1].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[1].set_title(f'After Movement\n (FN+FP): {loss_new} | (TP+TN): {firm_utility_moved} | (TP-FP) : {tp_accepted_moved-tp_lost_moved}')
        ax[1].set_xlim(x_min, x_max)
        ax[1].set_ylim(y_min, y_max)
        ax[1].set_xlabel('Feature 1')
        ax[1].set_ylabel('Feature 2')
        ax[1].legend()

        plt.tight_layout()
        # Save with mode in the name
        plt.savefig(f'{mode}_dist.png')
        plt.show()

    elif mode == 'B':
        if a > b:
            a_B = w(a, gamma)
            b_B = 1-a_B
        else:
            b_B = w(b, gamma)
            a_B = 1-b_B
    
        # Calculate initial loss
        tp_lost, tp_accepted, tn_accepted, tn_lost = compute_loss(X_0, X_1, a, b, t)
        firm_utility = tp_accepted - tn_accepted
        loss_initial = tp_lost + tn_lost

        # Movement region parameters
        mask_0 = (t - B <= a_B * X_0[:, 0] + b_B * X_0[:, 1]) & (a_B * X_0[:, 0] + b_B * X_0[:, 1] < t)
        mask_1 = (t - B <= a_B * X_1[:, 0] + b_B * X_1[:, 1]) & (a_B * X_1[:, 0] + b_B * X_1[:, 1] < t)

        # Move the distributions
        X_0_moved = X_0.copy()
        X_1_moved = X_1.copy()

        move_towards_boundary(X_0_moved, mask_0, a_B, b_B, t)
        move_towards_boundary(X_1_moved, mask_1, a_B, b_B, t)

        # Calculate new loss after movement
        tp_lost_moved, tp_accepted_moved, tn_accepted_moved, tn_lost_moved = compute_loss(X_0_moved, X_1_moved, a, b, t)
        firm_utility_moved = tp_accepted_moved - tn_accepted_moved
        loss_new = tp_lost_moved + tn_lost_moved
def plot_dist(X_0, X_1, a, b, t, B, mode='NB', gamma=0.6):
    if mode == 'NB':
        # Calculate initial loss
        tp_lost, tp_accepted, tn_accepted, tn_lost = compute_loss(X_0, X_1, a, b, t)
        firm_utility = tp_accepted - tn_accepted
        loss_initial = tp_lost + tn_lost

        # Movement region parameters
        mask_0 = (t - B <= a * X_0[:, 0] + b * X_0[:, 1]) & (a * X_0[:, 0] + b * X_0[:, 1] < t)
        mask_1 = (t - B <= a * X_1[:, 0] + b * X_1[:, 1]) & (a * X_1[:, 0] + b * X_1[:, 1] < t)

        # Move the distributions
        X_0_moved = X_0.copy()
        X_1_moved = X_1.copy()

        move_towards_boundary(X_0_moved, mask_0, a, b, t)
        move_towards_boundary(X_1_moved, mask_1, a, b, t)

        # Calculate new loss after movement
        tp_lost_moved, tp_accepted_moved, tn_accepted_moved, tn_lost_moved = compute_loss(X_0_moved, X_1_moved, a, b, t)
        firm_utility_moved = tp_accepted_moved - tn_accepted_moved
        loss_new = tp_lost_moved + tn_lost_moved

        # Create a grid of points to evaluate the KDE
        x_min, x_max = min(np.min(X_0[:, 0]), np.min(X_1[:, 0])) - 1, max(np.max(X_0[:, 0]), np.max(X_1[:, 0])) + 1
        y_min, y_max = min(np.min(X_0[:, 1]), np.min(X_1[:, 1])) - 1, max(np.max(X_0[:, 1]), np.max(X_1[:, 1])) + 1
        xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
        positions = np.vstack([xx.ravel(), yy.ravel()])

        # KDE for each class
        kde_0 = gaussian_kde(X_0.T)
        kde_1 = gaussian_kde(X_1.T)
        kde_0_moved = gaussian_kde(X_0_moved.T)
        kde_1_moved = gaussian_kde(X_1_moved.T)

        # Evaluate the KDE on the grid
        f_0 = np.reshape(kde_0(positions).T, xx.shape)
        f_1 = np.reshape(kde_1(positions).T, xx.shape)
        f_0_moved = np.reshape(kde_0_moved(positions).T, xx.shape)
        f_1_moved = np.reshape(kde_1_moved(positions).T, xx.shape)

        # Create the plot
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))

        # Plot 1: Initial Distribution KDE
        ax[0].contourf(xx, yy, f_0, cmap='Reds', alpha=0.4)
        ax[0].contourf(xx, yy, f_1, cmap='Blues', alpha=0.4)
        ax[0].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[0].set_title(f'Initial Distribution\n (FN+FP): {loss_initial} | (TP+TN): {firm_utility} | (TP-FP) : {tp_accepted-tp_lost}')
        ax[0].set_xlim(x_min, x_max)
        ax[0].set_ylim(y_min, y_max)
        ax[0].set_xlabel('Feature 1')
        ax[0].set_ylabel('Feature 2')
        ax[0].legend()

        # Plot 2: Distribution After Movement KDE
        ax[1].contourf(xx, yy, f_0_moved, cmap='Reds', alpha=0.4)
        ax[1].contourf(xx, yy, f_1_moved, cmap='Blues', alpha=0.4)
        ax[1].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[1].set_title(f'After Movement\n (FN+FP): {loss_new} | (TP+TN): {firm_utility_moved} | (TP-FP) : {tp_accepted_moved-tp_lost_moved}')
        ax[1].set_xlim(x_min, x_max)
        ax[1].set_ylim(y_min, y_max)
        ax[1].set_xlabel('Feature 1')
        ax[1].set_ylabel('Feature 2')
        ax[1].legend()

        plt.tight_layout()
        # Save with mode in the name
        plt.savefig(f'{mode}_dist.png')
        plt.show()

    elif mode == 'B':
        if a > b:
            a_B = w(a, gamma)
            b_B = 1-a_B
        else:
            b_B = w(b, gamma)
            a_B = 1-b_B
    
        # Calculate initial loss
        tp_lost, tp_accepted, tn_accepted, tn_lost = compute_loss(X_0, X_1, a, b, t)
        firm_utility = tp_accepted - tn_accepted
        loss_initial = tp_lost + tn_lost

        # Movement region parameters
        mask_0 = (t - B <= a_B * X_0[:, 0] + b_B * X_0[:, 1]) & (a_B * X_0[:, 0] + b_B * X_0[:, 1] < t)
        mask_1 = (t - B <= a_B * X_1[:, 0] + b_B * X_1[:, 1]) & (a_B * X_1[:, 0] + b_B * X_1[:, 1] < t)

        # Move the distributions
        X_0_moved = X_0.copy()
        X_1_moved = X_1.copy()

        move_towards_boundary(X_0_moved, mask_0, a_B, b_B, t)
        move_towards_boundary(X_1_moved, mask_1, a_B, b_B, t)

        # Calculate new loss after movement
        tp_lost_moved, tp_accepted_moved, tn_accepted_moved, tn_lost_moved = compute_loss(X_0_moved, X_1_moved, a, b, t)
        firm_utility_moved = tp_accepted_moved - tn_accepted_moved
        loss_new = tp_lost_moved + tn_lost_moved

        
def plot_dist(X_0, X_1, a, b, t, B, mode='NB', gamma=0.6):
    if mode == 'NB':
        # Calculate initial loss
        tp_lost, tp_accepted, tn_accepted, tn_lost = compute_loss(X_0, X_1, a, b, t)
        firm_utility = tp_accepted - tn_accepted
        loss_initial = tp_lost + tn_lost

        # Movement region parameters
        mask_0 = (t - B <= a * X_0[:, 0] + b * X_0[:, 1]) & (a * X_0[:, 0] + b * X_0[:, 1] < t)
        mask_1 = (t - B <= a * X_1[:, 0] + b * X_1[:, 1]) & (a * X_1[:, 0] + b * X_1[:, 1] < t)

        # Move the distributions
        X_0_moved = X_0.copy()
        X_1_moved = X_1.copy()

        move_towards_boundary(X_0_moved, mask_0, a, b, t)
        move_towards_boundary(X_1_moved, mask_1, a, b, t)

        # Calculate new loss after movement
        tp_lost_moved, tp_accepted_moved, tn_accepted_moved, tn_lost_moved = compute_loss(X_0_moved, X_1_moved, a, b, t)
        firm_utility_moved = tp_accepted_moved - tn_accepted_moved
        loss_new = tp_lost_moved + tn_lost_moved

        # Create a grid of points to evaluate the KDE
        x_min, x_max = min(np.min(X_0[:, 0]), np.min(X_1[:, 0])) - 1, max(np.max(X_0[:, 0]), np.max(X_1[:, 0])) + 1
        y_min, y_max = min(np.min(X_0[:, 1]), np.min(X_1[:, 1])) - 1, max(np.max(X_0[:, 1]), np.max(X_1[:, 1])) + 1
        xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
        positions = np.vstack([xx.ravel(), yy.ravel()])

        # KDE for each class
        kde_0 = gaussian_kde(X_0.T)
        kde_1 = gaussian_kde(X_1.T)
        kde_0_moved = gaussian_kde(X_0_moved.T)
        kde_1_moved = gaussian_kde(X_1_moved.T)

        # Evaluate the KDE on the grid
        f_0 = np.reshape(kde_0(positions).T, xx.shape)
        f_1 = np.reshape(kde_1(positions).T, xx.shape)
        f_0_moved = np.reshape(kde_0_moved(positions).T, xx.shape)
        f_1_moved = np.reshape(kde_1_moved(positions).T, xx.shape)

        # Create the plot
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))

        # Plot 1: Initial Distribution KDE
        ax[0].contourf(xx, yy, f_0, cmap='Reds', alpha=0.4)
        ax[0].contourf(xx, yy, f_1, cmap='Blues', alpha=0.4)
        ax[0].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[0].set_title(f'Initial Distribution\n (FN+FP): {loss_initial} | (TP+TN): {firm_utility} | (TP-FP) : {tp_accepted-tp_lost}')
        ax[0].set_xlim(x_min, x_max)
        ax[0].set_ylim(y_min, y_max)
        ax[0].set_xlabel('Feature 1')
        ax[0].set_ylabel('Feature 2')
        ax[0].legend()

        # Plot 2: Distribution After Movement KDE
        ax[1].contourf(xx, yy, f_0_moved, cmap='Reds', alpha=0.4)
        ax[1].contourf(xx, yy, f_1_moved, cmap='Blues', alpha=0.4)
        ax[1].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[1].set_title(f'After Movement\n (FN+FP): {loss_new} | (TP+TN): {firm_utility_moved} | (TP-FP) : {tp_accepted_moved-tp_lost_moved}')
        ax[1].set_xlim(x_min, x_max)
        ax[1].set_ylim(y_min, y_max)
        ax[1].set_xlabel('Feature 1')
        ax[1].set_ylabel('Feature 2')
        ax[1].legend()

        plt.tight_layout()
        # Save with mode in the name
        plt.savefig(f'{mode}_dist.png')
        plt.show()

    elif mode == 'B':
        if a > b:
            a_B = w(a, gamma)
            b_B = 1-a_B
        else:
            b_B = w(b, gamma)
            a_B = 1-b_B
    
        # Calculate initial loss
        tp_lost, tp_accepted, tn_accepted, tn_lost = compute_loss(X_0, X_1, a, b, t)
        firm_utility = tp_accepted - tn_accepted
        loss_initial = tp_lost + tn_lost

        # Movement region parameters
        mask_0 = (t - B <= a_B * X_0[:, 0] + b_B * X_0[:, 1]) & (a_B * X_0[:, 0] + b_B * X_0[:, 1] < t)
        mask_1 = (t - B <= a_B * X_1[:, 0] + b_B * X_1[:, 1]) & (a_B * X_1[:, 0] + b_B * X_1[:, 1] < t)

        # Move the distributions
        X_0_moved = X_0.copy()
        X_1_moved = X_1.copy()

        move_towards_boundary(X_0_moved, mask_0, a_B, b_B, t)
        move_towards_boundary(X_1_moved, mask_1, a_B, b_B, t)

        # Calculate new loss after movement
        tp_lost_moved, tp_accepted_moved, tn_accepted_moved, tn_lost_moved = compute_loss(X_0_moved, X_1_moved, a, b, t)
        firm_utility_moved = tp_accepted_moved - tn_accepted_moved
        loss_new = tp_lost_moved + tn_lost_moved

        # Create a grid of points to evaluate the KDE
        x_min, x_max = min(np.min(X_0[:, 0]), np.min(X_1[:, 0])) - 1, max(np.max(X_0[:, 0]), np.max(X_1[:, 0])) + 1
        y_min, y_max = min(np.min(X_0[:, 1]), np.min(X_1[:, 1])) - 1, max(np.max(X_0[:, 1]), np.max(X_1[:, 1])) + 1
        xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
        positions = np.vstack([xx.ravel(), yy.ravel()])

        # KDE for each class
        kde_0 = gaussian_kde(X_0.T)
        kde_1 = gaussian_kde(X_1.T)
        kde_0_moved = gaussian_kde(X_0_moved.T)
        kde_1_moved = gaussian_kde(X_1_moved.T)

        # Evaluate the KDE on the grid
        f_0 = np.reshape(kde_0(positions).T, xx.shape)
        f_1 = np.reshape(kde_1(positions).T, xx.shape)
        f_0_moved = np.reshape(kde_0_moved(positions).T, xx.shape)
        f_1_moved = np.reshape(kde_1_moved(positions).T, xx.shape)

        # Create the plot
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))

        # Plot 1: Initial Distribution KDE
        ax[0].contourf(xx, yy, f_0, cmap='Reds', alpha=0.4)
        ax[0].contourf(xx, yy, f_1, cmap='Blues', alpha=0.4)
        ax[0].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[0].plot(np.linspace(x_min, x_max, 100), (t - a_B * np.linspace(x_min, x_max, 100)) / b_B, color='gold', linestyle='--', label='Perceived Boundary')
        ax[0].set_title(f'Initial Distribution\n (FN+FP): {loss_initial} | (TP+TN): {firm_utility} | (TP-FP) : {tp_accepted-tp_lost}')
        ax[0].set_xlim(x_min, x_max)
        ax[0].set_ylim(y_min, y_max)
        ax[0].set_xlabel('Feature 1')
        ax[0].set_ylabel('Feature 2')
        ax[0].legend()

        # Plot 2: Distribution After Movement KDE
        ax[1].contourf(xx, yy, f_0_moved, cmap='Reds', alpha=0.4)
        ax[1].contourf(xx, yy, f_1_moved, cmap='Blues', alpha=0.4)
        ax[1].plot(np.linspace(x_min, x_max, 100), (t - a * np.linspace(x_min, x_max, 100)) / b, color='black', linestyle='--', label='Original Boundary')
        ax[1].plot(np.linspace(x_min, x_max, 100), (t - a_B * np.linspace(x_min, x_max, 100)) / b_B, color='gold', linestyle='--', label='Perceived Boundary')
        ax[1].set_title(f'After Movement\n (FN+FP): {loss_new} | (TP+TN): {firm_utility_moved} | (TP-FP) : {tp_accepted_moved-tp_lost_moved}')
        ax[1].set_xlim(x_min, x_max)
        ax[1].set_ylim(y_min, y_max)
        ax[1].set_xlabel('Feature 1')
        ax[1].set_ylabel('Feature 2')
        ax[1].legend()
        

        plt.tight_layout()
        # Save with mode in the name
        plt.savefig(f'{mode}_dist.png')
        plt.show()


def w(beta, gamma):
    """
    Applies the transformation to beta.
    
    Args:
    beta (float): The input weight.
    gamma (float): The gamma parameter for the transformation.

    Returns:
    float: The transformed weight.
    """
    return np.exp(-(-np.log(beta))**gamma)

def logistic_regression_with_sum_constraint(X, y):
    """
    Train logistic regression model with the constraint that coefficients sum to 1.
    Args:
    X (pd.DataFrame): Feature dataframe.
    y (pd.Series): Target vector.

    Returns:
    tuple: (coefficients, threshold)
    """
    n_samples, n_features = X.shape
    
    # Variables
    beta = cp.Variable(n_features)
    intercept = cp.Variable()

    # Logistic loss
    logits = X.values @ beta + intercept
    log_likelihood = cp.sum(
        cp.multiply(y.values, logits) - cp.logistic(logits)
    )

    # Objective and constraints
    objective = cp.Maximize(log_likelihood)
    constraints = [cp.sum(beta) == 1, 
                   beta >= 0, 
                   beta <= 1
                   ]

    # Problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    return beta.value, intercept.value, objective.value

In [3]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

def plot_dist_plotly(X_0, X_1, a, b, t, B, mode='NB', gamma=0.6):
    if mode == 'NB':
        # Calculate initial loss
        tp_lost, tp_accepted, tn_accepted, tn_lost = compute_loss(X_0, X_1, a, b, t)
        firm_utility = tp_accepted - tn_accepted
        loss_initial = tp_lost + tn_lost

        # Movement region parameters
        mask_0 = (t - B <= a * X_0[:, 0] + b * X_0[:, 1]) & (a * X_0[:, 0] + b * X_0[:, 1] < t)
        mask_1 = (t - B <= a * X_1[:, 0] + b * X_1[:, 1]) & (a * X_1[:, 0] + b * X_1[:, 1] < t)

        # Move the distributions
        X_0_moved_nb = X_0.copy()
        X_1_moved_nb = X_1.copy()

        move_towards_boundary(X_0_moved_nb, mask_0, a, b, t)
        move_towards_boundary(X_1_moved_nb, mask_1, a, b, t)

        # Calculate new loss after movement
        tp_lost_moved, tp_accepted_moved, tn_accepted_moved, tn_lost_moved = compute_loss(X_0_moved_nb, X_1_moved_nb, a, b, t)
        firm_utility_moved = tp_accepted_moved - tn_accepted_moved
        loss_new = tp_lost_moved + tn_lost_moved

        # Create a grid of points to evaluate the KDE
        x_min, x_max = min(np.min(X_0[:, 0]), np.min(X_1[:, 0])) - 1, max(np.max(X_0[:, 0]), np.max(X_1[:, 0])) + 1
        y_min, y_max = min(np.min(X_0[:, 1]), np.min(X_1[:, 1])) - 1, max(np.max(X_0[:, 1]), np.max(X_1[:, 1])) + 1
        xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
        positions = np.vstack([xx.ravel(), yy.ravel()])

        # KDE for each class
        kde_0 = gaussian_kde(X_0.T)
        kde_1 = gaussian_kde(X_1.T)
        kde_0_moved_nb = gaussian_kde(X_0_moved_nb.T)
        kde_1_moved_nb = gaussian_kde(X_1_moved_nb.T)

        # Evaluate the KDE on the grid
        f_0 = np.reshape(kde_0(positions).T, xx.shape)
        f_1 = np.reshape(kde_1(positions).T, xx.shape)
        f_0_moved_nb = np.reshape(kde_0_moved_nb(positions).T, xx.shape)
        f_1_moved_nb = np.reshape(kde_1_moved_nb(positions).T, xx.shape)

        # Create subplots
        fig = make_subplots(rows=1, cols=2, subplot_titles=(
            f'Initial Distribution<br>(FN+FP): {loss_initial} | (TP+TN): {firm_utility} | (TP-FP): {tp_accepted - tp_lost}',
            f'After Movement<br>(FN+FP): {loss_new} | (TP+TN): {firm_utility_moved} | (TP-FP): {tp_accepted_moved - tp_lost_moved}'
        ))

        # Plot 1: Initial Distribution KDE
        fig.add_trace(go.Contour(
            z=f_0.T,
            x=np.linspace(x_min, x_max, 100),
            y=np.linspace(y_min, y_max, 100),
            colorscale='Reds',
            opacity=0.4,
            showscale=False,
        ), row=1, col=1)

        fig.add_trace(go.Contour(
            z=f_1.T,
            x=np.linspace(x_min, x_max, 100),
            y=np.linspace(y_min, y_max, 100),
            colorscale='Blues',
            opacity=0.4,
            showscale=False,
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=np.linspace(x_min, x_max, 100),
            y=(t - a * np.linspace(x_min, x_max, 100)) / b,
            mode='lines',
            line=dict(color='black', dash='dash'),
            name='Original Boundary',
            showlegend=False
        ), row=1, col=1)

        # Plot 2: Distribution After Movement KDE
        fig.add_trace(go.Contour(
            z=f_0_moved_nb.T,
            x=np.linspace(x_min, x_max, 100),
            y=np.linspace(y_min, y_max, 100),
            colorscale='Reds',
            opacity=0.4,
            showscale=False,
        ), row=1, col=2)

        fig.add_trace(go.Contour(
            z=f_1_moved_nb.T,
            x=np.linspace(x_min, x_max, 100),
            y=np.linspace(y_min, y_max, 100),
            colorscale='Blues',
            opacity=0.4,
            showscale=False,
        ), row=1, col=2)

        fig.add_trace(go.Scatter(
            x=np.linspace(x_min, x_max, 100),
            y=(t - a * np.linspace(x_min, x_max, 100)) / b,
            mode='lines',
            line=dict(color='black', dash='dash'),
            name='Original Boundary'
        ), row=1, col=2)

        # Update layout
        fig.update_layout(
            showlegend=True,
            xaxis=dict(title='Feature 1', range=[x_min, x_max]),
            yaxis=dict(title='Feature 2', range=[y_min, y_max]),
            xaxis2=dict(title='Feature 1', range=[x_min, x_max]),
            yaxis2=dict(title='Feature 2', range=[y_min, y_max]),
            height=600,
            width=1200,
            template='plotly_dark'
        )

        fig.show()

    elif mode == 'B':
        if a > b:
            a_B = w(a, gamma)
            b_B = 1 - a_B
        else:
            b_B = w(b, gamma)
            a_B = 1 - b_B
        
        # Calculate initial loss
        tp_lost, tp_accepted, tn_accepted, tn_lost = compute_loss(X_0, X_1, a, b, t)
        firm_utility = tp_accepted - tn_accepted
        loss_initial = tp_lost + tn_lost

        # Movement region parameters
        mask_0 = (t - B <= a_B * X_0[:, 0] + b_B * X_0[:, 1]) & (a_B * X_0[:, 0] + b_B * X_0[:, 1] < t)
        mask_1 = (t - B <= a_B * X_1[:, 0] + b_B * X_1[:, 1]) & (a_B * X_1[:, 0] + b_B * X_1[:, 1] < t)

        # Move the distributions
        X_0_moved_b = X_0.copy()
        X_1_moved_b = X_1.copy()

        move_towards_boundary(X_0_moved_b, mask_0, a_B, b_B, t)
        move_towards_boundary(X_1_moved_b, mask_1, a_B, b_B, t)

        # Calculate new loss after movement
        tp_lost_moved, tp_accepted_moved, tn_accepted_moved, tn_lost_moved = compute_loss(X_0_moved_b, X_1_moved_b, a, b, t)
        firm_utility_moved = tp_accepted_moved - tn_accepted_moved
        loss_new = tp_lost_moved + tn_lost_moved

        # Create a grid of points to evaluate the KDE
        x_min, x_max = min(np.min(X_0[:, 0]), np.min(X_1[:, 0])) - 1, max(np.max(X_0[:, 0]), np.max(X_1[:, 0])) + 1
        y_min, y_max = min(np.min(X_0[:, 1]), np.min(X_1[:, 1])) - 1, max(np.max(X_0[:, 1]), np.max(X_1[:, 1])) + 1
        xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
        positions = np.vstack([xx.ravel(), yy.ravel()])

        # KDE for each class
        kde_0 = gaussian_kde(X_0.T)
        kde_1 = gaussian_kde(X_1.T)
        kde_0_moved_b = gaussian_kde(X_0_moved_b.T)
        kde_1_moved_b = gaussian_kde(X_1_moved_b.T)

        # Evaluate the KDE on the grid
        f_0 = np.reshape(kde_0(positions).T, xx.shape)
        f_1 = np.reshape(kde_1(positions).T, xx.shape)
        f_0_moved_b = np.reshape(kde_0_moved_b(positions).T, xx.shape)
        f_1_moved_b = np.reshape(kde_1_moved_b(positions).T, xx.shape)

        # Create subplots
        fig = make_subplots(rows=1, cols=2, subplot_titles=(
            f'Initial Distribution<br>(FN+FP): {loss_initial} | (TP+TN): {firm_utility} | (TP-FP): {tp_accepted - tp_lost}',
            f'After Movement<br>(FN+FP): {loss_new} | (TP+TN): {firm_utility_moved} | (TP-FP): {tp_accepted_moved - tp_lost_moved}'
        ))

        # Plot 1: Initial Distribution KDE
        fig.add_trace(go.Contour(
            z=f_0.T,
            x=np.linspace(x_min, x_max, 100),
            y=np.linspace(y_min, y_max, 100),
            colorscale='Reds',
            opacity=0.4,
            showscale=False,
        ), row=1, col=1)
        
        fig.add_trace(go.Contour(
            z=f_1.T,
            x=np.linspace(x_min, x_max, 100),
            y=np.linspace(y_min, y_max, 100),
            colorscale='Blues',
            opacity=0.4,
            showscale=False,
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=np.linspace(x_min, x_max, 100),
            y=(t - a * np.linspace(x_min, x_max, 100)) / b,
            mode='lines',
            line=dict(color='black', dash='dash'),
            name='Original Boundary',
            showlegend=False
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=np.linspace(x_min, x_max, 100),
            y=(t - a_B * np.linspace(x_min, x_max, 100)) / b_B,
            mode='lines',
            line=dict(color='gold', dash='dash'),
            name='Perceived Boundary',
            showlegend=False
        ), row=1, col=1)

        # Plot 2: Distribution After Movement KDE
        fig.add_trace(go.Contour(
            z=f_0_moved_b.T,
            x=np.linspace(x_min, x_max, 100),
            y=np.linspace(y_min, y_max, 100),
            colorscale='Reds',
            opacity=0.4,
            showscale=False,
        ), row=1, col=2)

        fig.add_trace(go.Contour(
            z=f_1_moved_b.T,
            x=np.linspace(x_min, x_max, 100),
            y=np.linspace(y_min, y_max, 100),
            colorscale='Blues',
            opacity=0.4,
            showscale=False,
        ), row=1, col=2)

        fig.add_trace(go.Scatter(
            x=np.linspace(x_min, x_max, 100),
            y=(t - a * np.linspace(x_min, x_max, 100)) / b,
            mode='lines',
            line=dict(color='black', dash='dash'),
            name='Original Boundary'
        ), row=1, col=2)

        fig.add_trace(go.Scatter(
            x=np.linspace(x_min, x_max, 100),
            y=(t - a_B * np.linspace(x_min, x_max, 100)) / b_B,
            mode='lines',
            line=dict(color='gold', dash='dash'),
            name='Perceived Boundary'
        ), row=1, col=2)

        # Update layout
        fig.update_layout(
            showlegend=True,
            xaxis=dict(title='Feature 1', range=[x_min, x_max]),
            yaxis=dict(title='Feature 2', range=[y_min, y_max]),
            xaxis2=dict(title='Feature 1', range=[x_min, x_max]),
            yaxis2=dict(title='Feature 2', range=[y_min, y_max]),
            title_text=f'Distribution Plot ({mode} mode)',
            height=600,
            width=1200,
            template='plotly_dark'
        )

        fig.show()

In [4]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

def plot_dist_all(X_0, X_1, a, b, t, B, gamma=0.6):    
    # Calculate initial loss
    fn, tp, fp, tn = compute_loss(X_0, X_1, a, b, t)
    trues_init = tp + tn
    misclassification_init = fn + fp
    firm_utility_init = tp - fp
    firm_util_2_init = tp - fp + tn - fn

    # Create a grid of points to evaluate the KDE
    x_min, x_max = min(np.min(X_0[:, 0]), np.min(X_1[:, 0])) - 1, max(np.max(X_0[:, 0]), np.max(X_1[:, 0])) + 1
    y_min, y_max = min(np.min(X_0[:, 1]), np.min(X_1[:, 1])) - 1, max(np.max(X_0[:, 1]), np.max(X_1[:, 1])) + 1
    xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])

    # KDE for each class
    kde_0 = gaussian_kde(X_0.T)
    kde_1 = gaussian_kde(X_1.T)
    f_0 = np.reshape(kde_0(positions).T, xx.shape)
    f_1 = np.reshape(kde_1(positions).T, xx.shape)

    # Original Distributions (No Movement)
    f_0_original = f_0
    f_1_original = f_1

    # NB Mode
    X_0_moved_NB = X_0.copy()
    X_1_moved_NB = X_1.copy()
    mask_0_NB = (t - B <= a * X_0[:, 0] + b * X_0[:, 1]) & (a * X_0[:, 0] + b * X_0[:, 1] < t)
    mask_1_NB = (t - B <= a * X_1[:, 0] + b * X_1[:, 1]) & (a * X_1[:, 0] + b * X_1[:, 1] < t)
    move_towards_boundary(X_0_moved_NB, mask_0_NB, a, b, t)
    move_towards_boundary(X_1_moved_NB, mask_1_NB, a, b, t)
    kde_0_moved_NB = gaussian_kde(X_0_moved_NB.T)
    kde_1_moved_NB = gaussian_kde(X_1_moved_NB.T)
    f_0_moved_NB = np.reshape(kde_0_moved_NB(positions).T, xx.shape)
    f_1_moved_NB = np.reshape(kde_1_moved_NB(positions).T, xx.shape)
    
    # B Mode
    if a > b:
        a_B = w(a, gamma)
        b_B = 1 - a_B
    else:
        b_B = w(b, gamma)
        a_B = 1 - b_B
    X_0_moved_B = X_0.copy()
    X_1_moved_B = X_1.copy()
    mask_0_B = (t - B <= a_B * X_0[:, 0] + b_B * X_0[:, 1]) & (a_B * X_0[:, 0] + b_B * X_0[:, 1] < t)
    mask_1_B = (t - B <= a_B * X_1[:, 0] + b_B * X_1[:, 1]) & (a_B * X_1[:, 0] + b_B * X_1[:, 1] < t)
    move_towards_boundary(X_0_moved_B, mask_0_B, a_B, b_B, t)
    move_towards_boundary(X_1_moved_B, mask_1_B, a_B, b_B, t)
    kde_0_moved_B = gaussian_kde(X_0_moved_B.T)
    kde_1_moved_B = gaussian_kde(X_1_moved_B.T)
    f_0_moved_B = np.reshape(kde_0_moved_B(positions).T, xx.shape)
    f_1_moved_B = np.reshape(kde_1_moved_B(positions).T, xx.shape)

    # Calculate new loss after movement NB
    fn_moved_nb, tp_moved_nb, fp_moved_nb, tn_moved_nb = compute_loss(X_0_moved_NB, X_1_moved_NB, a, b, t)
    trues_moved_nb = tp_moved_nb + tn_moved_nb
    misclassification_moved_nb = fn_moved_nb + fp_moved_nb
    firm_utility_moved_nb = tp_moved_nb - fp_moved_nb
    firm_util_2_nb = tp_moved_nb - fp_moved_nb + tn_moved_nb - fn_moved_nb


    # Calculate new loss after movement B
    fn_moved_b, tp_moved_b, fp_moved_b, tn_moved_b = compute_loss(X_0_moved_B, X_1_moved_B, a, b, t)
    trues_moved_b = tp_moved_b + tn_moved_b
    misclassification_moved_b = fn_moved_b + fp_moved_b
    firm_utility_moved_b = tp_moved_b - fp_moved_b
    firm_util_2_b = tp_moved_b - fp_moved_b + tn_moved_b - fn_moved_b

    # Create subplots with 3 columns
    fig = make_subplots(rows=1, cols=3, subplot_titles=(
            # f'Initial Distribution<br>(TP-FP): {tp} - {fp} = {firm_utility_init}',
            # f'Non-biased strategic response<br>(TP-FP): {tp_moved_nb} - {fp_moved_nb} = {firm_utility_moved_nb}',
            # f'Biased strategic response<br>(TP-FP): {tp_moved_b} - {fp_moved_b} = {firm_utility_moved_b}'
            f'Initial Distribution<br>(TP+TN-FP-FN): {firm_util_2_init}',
            f'Non-biased strategic response<br>(TP+TN-FP-FN): {firm_util_2_nb}',
            f'Biased strategic response<br>(TP+TN-FP-FN): {firm_util_2_b}'
        ))

    # Plot 1: Original Distribution
    fig.add_trace(go.Contour(
        z=f_0_original.T,
        x=np.linspace(x_min, x_max, 100),
        y=np.linspace(y_min, y_max, 100),
        colorscale='Reds',
        opacity=0.4,
        showscale=False,
    ), row=1, col=1)

    fig.add_trace(go.Contour(
        z=f_1_original.T,
        x=np.linspace(x_min, x_max, 100),
        y=np.linspace(y_min, y_max, 100),
        colorscale='Blues',
        opacity=0.4,
        showscale=False,
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=np.linspace(x_min, x_max, 100),
        y=(t - a * np.linspace(x_min, x_max, 100)) / b,
        mode='lines',
        line=dict(color='black'),
        name='Original Boundary',
        showlegend=False
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=np.linspace(x_min, x_max, 100),
        y=(t - B - a * np.linspace(x_min, x_max, 100)) / b,
        mode='lines',
        line=dict(color='black', dash='dash'),
        name='Original Boundary Budget',
        showlegend=False
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=np.linspace(x_min, x_max, 100),
        y=(t - a_B * np.linspace(x_min, x_max, 100)) / b_B,
        mode='lines',
        line=dict(color='gold'),
        name='Perceived Boundary',
        showlegend=False
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=np.linspace(x_min, x_max, 100),
        y=(t - B - a_B * np.linspace(x_min, x_max, 100)) / b_B,
        mode='lines',
        line=dict(color='gold', dash='dash'),
        name='Perceived Boundary Budget',
        showlegend=False
    ), row=1, col=1)


    # Plot 2: NB Mode
    fig.add_trace(go.Contour(
        z=f_0_moved_NB.T,
        x=np.linspace(x_min, x_max, 100),
        y=np.linspace(y_min, y_max, 100),
        colorscale='Reds',
        opacity=0.4,
        showscale=False,
    ), row=1, col=2)

    fig.add_trace(go.Contour(
        z=f_1_moved_NB.T,
        x=np.linspace(x_min, x_max, 100),
        y=np.linspace(y_min, y_max, 100),
        colorscale='Blues',
        opacity=0.4,
        showscale=False,
    ), row=1, col=2)

    fig.add_trace(go.Scatter(
        x=np.linspace(x_min, x_max, 100),
        y=(t - a * np.linspace(x_min, x_max, 100)) / b,
        mode='lines',
        line=dict(color='black'),
        name='Original Boundary',
        showlegend=False
    ), row=1, col=2)

    fig.add_trace(go.Scatter(
        x=np.linspace(x_min, x_max, 100),
        y=(t - B - a * np.linspace(x_min, x_max, 100)) / b,
        mode='lines',
        line=dict(color='black', dash='dash'),
        name='Original Boundary',
        showlegend=False
    ), row=1, col=2)


    # Plot 3: B Mode
    fig.add_trace(go.Contour(
        z=f_0_moved_B.T,
        x=np.linspace(x_min, x_max, 100),
        y=np.linspace(y_min, y_max, 100),
        colorscale='Reds',
        opacity=0.4,
        showscale=False,
    ), row=1, col=3)

    fig.add_trace(go.Contour(
        z=f_1_moved_B.T,
        x=np.linspace(x_min, x_max, 100),
        y=np.linspace(y_min, y_max, 100),
        colorscale='Blues',
        opacity=0.4,
        showscale=False,
    ), row=1, col=3)

    fig.add_trace(go.Scatter(
        x=np.linspace(x_min, x_max, 100),
        y=(t - a * np.linspace(x_min, x_max, 100)) / b,
        mode='lines',
        line=dict(color='black'),
        name='Original',
    ), row=1, col=3)

    fig.add_trace(go.Scatter(
        x=np.linspace(x_min, x_max, 100),
        y=(t - B - a * np.linspace(x_min, x_max, 100)) / b,
        mode='lines',
        line=dict(color='black', dash='dash'),
        name='Budget',
        # showlegend=False
    ), row=1, col=3)

    fig.add_trace(go.Scatter(
        x=np.linspace(x_min, x_max, 100),
        y=(t - a_B * np.linspace(x_min, x_max, 100)) / b_B,
        mode='lines',
        line=dict(color='gold'),
        name='Perceived',
    ), row=1, col=3)

    fig.add_trace(go.Scatter(
        x=np.linspace(x_min, x_max, 100),
        y=(t - B - a_B * np.linspace(x_min, x_max, 100)) / b_B,
        mode='lines',
        line=dict(color='gold', dash='dash'),
        name='Budget',
    ), row=1, col=3)


    # Update layout
    fig.update_layout(
        showlegend=True,
        font=dict(size=20),  # Increase the font size for the entire plot
        xaxis=dict(title='Feature 1', range=[x_min, x_max]),
        yaxis=dict(title='Feature 2', range=[y_min, y_max]),
        xaxis2=dict(title='Feature 1', range=[x_min, x_max]),
        yaxis2=dict(range=[y_min, y_max]),
        xaxis3=dict(title='Feature 1', range=[x_min, x_max]),
        yaxis3=dict(range=[y_min, y_max]),
        height=600,
        width=1800,
        template='presentation'
    )

    # Update the font size for the subplot titles
    for annotation in fig['layout']['annotations']:
        annotation['font'] = dict(size=24)  # Increase font size for subplot titles

    fig.show()

In [5]:
def count_points_in_custom_regions(X_0, X_1, a, b, t, B, gamma=0.6):
    """
    Count the number of points in two specified regions and their intersection for both Class 0 and Class 1.

    Parameters:
    - X_0: numpy array of shape (n_samples, 2) for Class 0 points
    - X_1: numpy array of shape (n_samples, 2) for Class 1 points
    - a, b, t: coefficients for the first linear boundary ax_1 + bx_2 = t
    - B: distance to shift the boundary
    - a_B, b_B: coefficients for the second linear boundary a_Bx_1 + b_Bx_2 = t
    
    Returns:
    - counts_0: Number of Class 0 points in each region and their intersection
    - counts_1: Number of Class 1 points in each region and their intersection
    """
    if a > b:
        a_B = w(a, gamma)
        b_B = 1 - a_B
    else:
        b_B = w(b, gamma)
        a_B = 1 - b_B

    # Region 1: t - B <= a * X[:, 0] + b * X[:, 1] < t
    mask_0_region1 = (t - B <= a * X_0[:, 0] + b * X_0[:, 1]) & (a * X_0[:, 0] + b * X_0[:, 1] < t)
    mask_1_region1 = (t - B <= a * X_1[:, 0] + b * X_1[:, 1]) & (a * X_1[:, 0] + b * X_1[:, 1] < t)

    # Region 2: t - B <= a_B * X[:, 0] + b_B * X[:, 1] < t
    mask_0_region2 = (t - B <= a_B * X_0[:, 0] + b_B * X_0[:, 1]) & (a_B * X_0[:, 0] + b_B * X_0[:, 1] < t)
    mask_1_region2 = (t - B <= a_B * X_1[:, 0] + b_B * X_1[:, 1]) & (a_B * X_1[:, 0] + b_B * X_1[:, 1] < t)

    # Intersection of Region 1 and Region 2
    mask_0_intersection = mask_0_region1 & mask_0_region2
    mask_1_intersection = mask_1_region1 & mask_1_region2

    # Movement
    X_0_moved_nb = X_0.copy()
    X_1_moved_nb = X_1.copy()
    move_towards_boundary(X_0_moved_nb, mask_0_region1, a, b, t)
    move_towards_boundary(X_1_moved_nb, mask_1_region1, a, b, t)

    X_0_moved_b = X_0.copy()
    X_1_moved_b = X_1.copy()
    move_towards_boundary(X_0_moved_b, mask_0_region2, a_B, b_B, t)
    move_towards_boundary(X_1_moved_b, mask_1_region2, a_B, b_B, t)

    # Above threshold after movement
    mask_0_region2_moved = (t <= a_B * X_0_moved_b[:, 0] + b_B * X_0_moved_b[:, 1]) & mask_0_intersection
    mask_1_region2_moved = (t <= a_B * X_1_moved_b[:, 0] + b_B * X_1_moved_b[:, 1]) & mask_1_intersection

    # Count the points in each region and their intersection
    count_0_region1 = np.sum(mask_0_region1)
    count_1_region1 = np.sum(mask_1_region1)
    count_0_region2 = np.sum(mask_0_region2)
    count_1_region2 = np.sum(mask_1_region2)
    count_0_intersection = np.sum(mask_0_intersection)
    count_1_intersection = np.sum(mask_1_intersection)
    count_0_region2_moved = np.sum(mask_0_region2_moved)
    count_1_region2_moved = np.sum(mask_1_region2_moved)

    return {
        "Class 0": {
            "Region NB": count_0_region1,
            "Region B": count_0_region2,
            "Intersection": count_0_intersection,
            "Behavioral agents that made it anyways": count_0_region2_moved
        },
        "Class 1": {
            "Region NB": count_1_region1,
            "Region B": count_1_region2,
            "Intersection": count_1_intersection,
            "Behavioral agents that made it anyways": count_1_region2_moved
        }
    }

In [16]:
# -----------------------------------------------------------------------------------------------
# Alternate plots:

# plot_for(X_0, X_1, a, b, t, B, mode='NB')
# plot_dist(X_0, X_1, a, b, t, B, mode='NB')

# plot_for(X_0, X_1, a, b, t, B, mode='B', gamma=0.3)
# plot_dist(X_0, X_1, a, b, t, B, mode='B', gamma=0.3)

# plot_dist_plotly(X_0, X_1, a, b, t, B, mode='NB')
# plot_dist_plotly(X_0, X_1, a, b, t, B, mode='B', gamma=0.3)
# -----------------------------------------------------------------------------------------------

# Set up the feature space
np.random.seed(42)
n_samples = 20000



# # Generate samples for class 0 - TWO DISTRIBUTIONS
# mean_0a = [4, 1]
# cov_0a = [[0.5, 0], [0, 0.5]]

# mean_0b = [1, 2]
# cov_0b = [[0.5, 0], [0, 0.5]]

# X_0a = np.random.multivariate_normal(mean_0a, cov_0a, n_samples // 2)/0.1
# X_0b = np.random.multivariate_normal(mean_0b, cov_0b, n_samples // 2)/0.1
# X_0 = np.vstack((X_0a, X_0b))

# # Generate samples for class 1 - SINGLE DISTRIBUTION
# mean_1 = [4, 4]
# cov_1 = [[1, 0], [0, 0.5]]
# X_1 = np.random.multivariate_normal(mean_1, cov_1, n_samples)/0.1

# Generate samples for class 1 - TWO DISTRIBUTIONS
# mean_1a = [5, 3]
# cov_1a = [[0.5, 0], [0, 0.5]]

# Generate samples for classe 0
mean_0 = [2, 3]
cov_0 = [[3, 0], [0, 1]]
X_0 = np.random.multivariate_normal(mean_0, cov_0, n_samples)/0.1

mean_1b = [2, 5] #[2, 4] and [3, 5]
cov_1b = [[1, 0], [0, 1]]

# X_1a = np.random.multivariate_normal(mean_1a, cov_1a, n_samples // 2)/0.1
X_1 = np.random.multivariate_normal(mean_1b, cov_1b, n_samples)/0.1
# X_1 = np.vstack((X_1a, X_1b))


# Find the best decision boundary using LR
X = np.concatenate([X_0, X_1])
y = np.concatenate([np.zeros(n_samples), np.ones(n_samples)])

beta, intercept, obj = logistic_regression_with_sum_constraint(pd.DataFrame(X), pd.Series(y))

print(f'Coefficients: {beta}')

a, b = beta
t = -intercept


B = 50  # Width of the region

counts = count_points_in_custom_regions(X_0, X_1, a, b, t, B, gamma=0.3)

print(f"Counts for Class 0: {counts['Class 0']}")
print(f"Counts for Class 1: {counts['Class 1']}")

plot_dist_all(X_0, X_1, a, b, t, B, gamma=0.3)


Coefficients: [0.13172304 0.86827696]
Counts for Class 0: {'Region NB': 16717, 'Region B': 17852, 'Intersection': 16092, 'Behavioral agents that made it anyways': 15031}
Counts for Class 1: {'Region NB': 3291, 'Region B': 10250, 'Intersection': 3266, 'Behavioral agents that made it anyways': 3002}


In [37]:
# # Find the best decision boundary on the transformed space
# a_B = w(a, 0.3)
# b_B = 1 - a_B

# # Movement region parameters
# mask_0 = (t - B <= a_B * X_0[:, 0] + b_B * X_0[:, 1]) & (a_B * X_0[:, 0] + b_B * X_0[:, 1] < t)
# mask_1 = (t - B <= a_B * X_1[:, 0] + b_B * X_1[:, 1]) & (a_B * X_1[:, 0] + b_B * X_1[:, 1] < t)

# # Move the distributions
# X_0_moved = X_0.copy()
# X_1_moved = X_1.copy()

# move_towards_boundary(X_0_moved, mask_0, a_B, b_B, t)
# move_towards_boundary(X_1_moved, mask_1, a_B, b_B, t)

# X_moved = np.concatenate([X_0_moved, X_1_moved])
# y_moved = np.concatenate([np.zeros(n_samples), np.ones(n_samples)])

# beta_moved, intercept_moved, obj_moved = logistic_regression_with_sum_constraint(pd.DataFrame(X_moved), pd.Series(y_moved))

# print(f'Objective before movement: {obj}')
# print(f'Objective after movement: {obj_moved}')

# print(f'Coefficients: {beta_moved}')

# a_moved, b_moved = beta_moved

# plot_for(X_0_moved, X_1_moved, a_moved, b_moved, t, B, mode='B')
# plot_dist(X_0_moved, X_1_moved, a_moved, b_moved, t, B, mode='B')